# Mathematical optimization: finding minima of functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import optimize
import collections
import sys
import os
sys.path.append(os.path.abspath("helper"))
from cost_functions import (
    mk_quad,
    mk_gauss,
    rosenbrock,
    rosenbrock_prime,
    rosenbrock_hessian,
    LoggingFunction,
    CountingFunction,
)

# A custom function for plotting.
def get_subplot_n(index):
    row = index+1
    if row == 1:
        subplot_n0 = 1
        subplot_n1 = 2
        subplot_n2 = 3
    elif row == 2:
        subplot_n0 = 4
        subplot_n1 = 5
        subplot_n2 = 6
    elif row == 3:
        subplot_n0 = 7
        subplot_n1 = 8
        subplot_n2 = 9
    elif row == 4:
        subplot_n0 = 10
        subplot_n1 = 11
        subplot_n2 = 12
    return subplot_n0, subplot_n1, subplot_n2

# Functions for gradient descent

###############################################################################
# A formatter to print values on contours
def super_fmt(value):
    if value > 1:
        if np.abs(int(value) - value) < 0.1:
            out = f"$10^{{{int(value):d}}}$"
        else:
            out = f"$10^{{{value:.1f}}}$"
    else:
        value = np.exp(value - 0.01)
        if value > 0.1:
            out = f"{value:1.1f}"
        elif value > 0.01:
            out = f"{value:.2f}"
        else:
            out = f"{value:.2e}"
    return out


###############################################################################
# A gradient descent algorithm
# do not use: its a toy, use scipy's optimize.fmin_cg

def gradient_descent(x0, f, f_prime, hessian=None, adaptative=False):
    x_i, y_i = x0
    all_x_i = []
    all_y_i = []
    all_f_i = []

    for i in range(1, 100):
        all_x_i.append(x_i)
        all_y_i.append(y_i)
        all_f_i.append(f([x_i, y_i]))
        dx_i, dy_i = f_prime(np.asarray([x_i, y_i]))
        if adaptative:
            # Compute a step size using a line_search to satisfy the Wolf
            # conditions
            step = sp.optimize.line_search(
                f,
                f_prime,
                np.r_[x_i, y_i],
                -np.r_[dx_i, dy_i],
                np.r_[dx_i, dy_i],
                c2=0.05,
            )
            step = step[0]
            if step is None:
                step = 0
        else:
            step = 1
        x_i += -step * dx_i
        y_i += -step * dy_i
        if np.abs(all_f_i[-1]) < 1e-16:
            break
    return all_x_i, all_y_i, all_f_i

def gradient_descent_adaptative(x0, f, f_prime, hessian=None):
    return gradient_descent(x0, f, f_prime, adaptative=True)

def conjugate_gradient(x0, f, f_prime, hessian=None):
    all_x_i = [x0[0]]
    all_y_i = [x0[1]]
    all_f_i = [f(x0)]

    def store(X):
        x, y = X
        all_x_i.append(x)
        all_y_i.append(y)
        all_f_i.append(f(X))

    sp.optimize.minimize(
        f, x0, jac=f_prime, method="CG", callback=store, options={"gtol": 1e-12}
    )
    return all_x_i, all_y_i, all_f_i


def newton_cg(x0, f, f_prime, hessian):
    all_x_i = [x0[0]]
    all_y_i = [x0[1]]
    all_f_i = [f(x0)]

    def store(X):
        x, y = X
        all_x_i.append(x)
        all_y_i.append(y)
        all_f_i.append(f(X))

    sp.optimize.minimize(
        f,
        x0,
        method="Newton-CG",
        jac=f_prime,
        hess=hessian,
        callback=store,
        options={"xtol": 1e-12},
    )
    return all_x_i, all_y_i, all_f_i


def bfgs(x0, f, f_prime, hessian=None):
    all_x_i = [x0[0]]
    all_y_i = [x0[1]]
    all_f_i = [f(x0)]

    def store(X):
        x, y = X
        all_x_i.append(x)
        all_y_i.append(y)
        all_f_i.append(f(X))

    sp.optimize.minimize(
        f, x0, method="BFGS", jac=f_prime, callback=store, options={"gtol": 1e-12}
    )
    return all_x_i, all_y_i, all_f_i


def powell(x0, f, f_prime, hessian=None):
    all_x_i = [x0[0]]
    all_y_i = [x0[1]]
    all_f_i = [f(x0)]

    def store(X):
        x, y = X
        all_x_i.append(x)
        all_y_i.append(y)
        all_f_i.append(f(X))

    sp.optimize.minimize(
        f, x0, method="Powell", callback=store, options={"ftol": 1e-12}
    )
    return all_x_i, all_y_i, all_f_i


def nelder_mead(x0, f, f_prime, hessian=None):
    all_x_i = [x0[0]]
    all_y_i = [x0[1]]
    all_f_i = [f(x0)]

    def store(X):
        x, y = X
        all_x_i.append(x)
        all_y_i.append(y)
        all_f_i.append(f(X))

    sp.optimize.minimize(
        f, x0, method="Nelder-Mead", callback=store, options={"ftol": 1e-12}
    )
    return all_x_i, all_y_i, all_f_i

**Authors**: *Gaël Varoquaux*

[Mathematical optimization](https://en.wikipedia.org/wiki/Mathematical_optimization) deals with the
problem of finding numerically minimums (or maximums or zeros) of
a function. In this context, the function is called *cost function*, or
*objective function*, or *energy*.

Here, we are interested in using {mod}`scipy.optimize` for black-box
optimization: we do not rely on the mathematical expression of the
function that we are optimizing. Note that this expression can often be
used for more efficient, non black-box, optimization.

**Start of admonition: Prerequisites**

 * {ref}`NumPy <numpy>`
 * {ref}`SciPy <scipy>`
 * {ref}`Matplotlib <matplotlib>`

**End of admonition**

**Start of admonition: See also**

**References**

Mathematical optimization is very ... mathematical. If you want
performance, it really pays to read the books:

- [Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/)
  by Boyd and Vandenberghe (pdf available free online).
- [Numerical Optimization](https://users.eecs.northwestern.edu/~nocedal/book/num-opt.html),
  by Nocedal and Wright. Detailed reference on gradient descent methods.
- [Practical Methods of Optimization](https://www.amazon.com/gp/product/0471494631/ref=ox_sc_act_title_1?ie=UTF8&smid=ATVPDKIKX0DER) by Fletcher: good at hand-waving explanations.
**End of admonition**

<!---
XXX: should I discuss root finding?
-->

## Knowing your problem

Not all optimization problems are equal. Knowing your problem enables you
to choose the right tool.

**Start of admonition: Dimensionality of the problem**
The scale of an optimization problem is pretty much set by the
*dimensionality of the problem*, i.e. the number of scalar variables
on which the search is performed.
**End of admonition**

### Convex versus non-convex optimization

In [ ]:
x = np.linspace(-1, 2)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)

# A convex function
plt.plot(x, x**2, linewidth=2)
plt.text(-0.7, -(0.6**2), "$f$", size=20)

# The tangent in one point
plt.plot(x, 2 * x - 1)
plt.plot(1, 1, "k+")
plt.text(0.3, -0.75, "Tangent to $f$", size=15)
plt.text(1, 1 - 0.5, "C", size=15)

# Convexity as barycenter
plt.plot([0.35, 1.85], [0.35**2, 1.85**2])
plt.plot([0.35, 1.85], [0.35**2, 1.85**2], "k+")
plt.text(0.35 - 0.2, 0.35**2 + 0.1, "A", size=15)
plt.text(1.85 - 0.2, 1.85**2, "B", size=15)

plt.ylim(ymin=-1)
plt.xticks([])
plt.yticks([])
plt.title("A Convex Function", fontstyle='italic')

# Convexity as barycenter
plt.subplot(1, 2, 2)
plt.plot(x, x**2 + np.exp(-5 * (x - 0.5) ** 2), linewidth=2)
plt.text(-0.7, -(0.6**2), "$f$", size=20)

plt.ylim(ymin=-1)
plt.xticks([])
plt.yticks([])
plt.title('A Non-convex Function', fontstyle='italic')

caption_text_1="""
- $f$ is above all its tangents.
- equivalently, for two points $A, B, f(C)$ lies below the segment
$[f(A), f(B])], \\text{if } A < C < B $
"""

plt.figtext(0.25, -0.2, caption_text_1, wrap=True,
            horizontalalignment='center',
            fontsize=12)
plt.tight_layout();

**Optimizing convex functions is easy. Optimizing non-convex functions can
be very hard.**

**Start of note**
It can be proven that for a convex function a local minimum is
also a global minimum. Then, in some sense, the minimum is unique.
**End of note**

### Smooth and non-smooth problems

In [ ]:
plt.figure(figsize=(8, 4))
x = np.linspace(-1.5, 1.5, 101)

# A smooth function
plt.subplot(1, 2, 1)

plt.plot(x, np.sqrt(0.2 + x**2), linewidth=2)
plt.text(-1, 0, "$f$", size=20)
plt.title('A Smooth Function',  fontstyle='italic')

plt.ylim(ymin=-0.2)
plt.axis("off")
plt.tight_layout()

# A non-smooth function
plt.subplot(1, 2, 2)
plt.plot(x, np.abs(x), linewidth=2)
plt.text(-1, 0, "$f$", size=20)
plt.title('A Non-smooth Function',  fontstyle='italic')

plt.ylim(ymin=-0.2)
caption_text_1="""
The gradient is defined everywhere, and is a continuous function.
"""

plt.figtext(0.25, -0.2, caption_text_1, wrap=True,
            horizontalalignment='center',
            fontsize=12)
plt.tight_layout();
plt.axis("off")
plt.tight_layout()

**Optimizing smooth functions is easier**
(true in the context of *black-box* optimization, otherwise
[Linear Programming](https://en.wikipedia.org/wiki/Linear_programming)
is an example of methods which deal very efficiently with
piece-wise linear functions).

### Noisy versus exact cost functions

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
caption_text_1="""
 Noisy (blue) and non-noisy (green) functions
"""

plt.text(0.3, 0.45, caption_text_1, wrap=True,
            horizontalalignment='left',
            fontsize=12)
plt.axis('off')

rng = np.random.default_rng(27446968)

x = np.linspace(-5, 5, 101)
x_ = np.linspace(-5, 5, 31)


def f(x):
    return -np.exp(-(x**2))


# A smooth function
plt.subplot(1, 2, 2)
plt.plot(x_, f(x_) + 0.2 * np.random.normal(size=31), linewidth=2)
plt.plot(x, f(x), linewidth=2)

plt.ylim(ymin=-1.3)
plt.axis("off")
plt.tight_layout()

**Start of admonition: Noisy gradients**
Many optimization methods rely on gradients of the objective function.
If the gradient function is not given, they are computed numerically,
which induces errors. In such situation, even if the objective
function is not noisy, a gradient-based optimization may be a noisy
optimization.
**End of admonition**

### Constraints

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
caption_text_1="""
 - Optimizations under constraints

    Here:

     $-1 < x_1 < 1$

     $-1 < x_2 < 1$
"""

plt.text(0.3, 0.45, caption_text_1, wrap=True,
            horizontalalignment='left',
            fontsize=12)
plt.axis('off')


plt.subplot(1, 2, 2)

x, y = np.mgrid[-2.9:5.8:0.05, -2.5:5:0.05]  # type: ignore[misc]
x = x.T
y = y.T

for i in (1, 2):
    # Create 2 figure: only the second one will have the optimization
    # path
    contours = plt.contour(
        np.sqrt((x - 3) ** 2 + (y - 2) ** 2),
        extent=[-3, 6, -2.5, 5],
        cmap="gnuplot",
    )
    plt.clabel(contours, inline=1, fmt="%1.1f", fontsize=14)
    plt.plot(
        [-1.5, -1.5, 1.5, 1.5, -1.5], [-1.5, 1.5, 1.5, -1.5, -1.5], "k", linewidth=2
    )
    plt.fill_between([-1.5, 1.5], [-1.5, -1.5], [1.5, 1.5], color=".8")
    plt.axvline(0, color="k")
    plt.axhline(0, color="k")

    plt.text(-0.9, 4.4, "$x_2$", size=20)
    plt.text(5.6, -0.6, "$x_1$", size=20)
    plt.axis("scaled")
    plt.axis("off")

# And now plot the optimization path
accumulator = []


def f(x):
    # Store the list of function calls
    accumulator.append(x)
    return np.sqrt((x[0] - 3) ** 2 + (x[1] - 2) ** 2)


# We don't use the gradient, as with the gradient, L-BFGS is too fast,
# and finds the optimum without showing us a pretty path
def f_prime(x):
    r = np.sqrt((x[0] - 3) ** 2 + (x[0] - 2) ** 2)
    return np.array(((x[0] - 3) / r, (x[0] - 2) / r))


sp.optimize.minimize(
    f, np.array([0, 0]), method="L-BFGS-B", bounds=((-1.5, 1.5), (-1.5, 1.5))
)
accumulated = np.array(accumulator)
plt.plot(accumulated[:, 0], accumulated[:, 1]);

## A review of the different optimizers

### Getting started: 1D optimization

Let's get started by finding the minimum of the scalar function
$f(x)=\exp[(x-0.5)^2]$. {func}`scipy.optimize.minimize_scalar` uses
Brent's method to find the minimum of a function:

In [ ]:
import numpy as np
import scipy as sp
def f(x):
    return -np.exp(-(x - 0.5)**2)
result = sp.optimize.minimize_scalar(f)
result.success # check if solver was successful

In [ ]:
x_min = result.x
x_min

In [ ]:
x_min - 0.5

In [ ]:
x = np.linspace(-1, 3, 100)
x_0 = np.exp(-1)

def f(x):
    return (x - x_0)**2 + epsilon*np.exp(-5*(x - .5 - x_0)**2)

plt.figure(figsize=(12, 6))
for epsilon in (0, 1):
    if epsilon == 0:
        subplot_n0 = 1
        subplot_n1 = 2
        subplot_n2 = 3
    else:
        subplot_n0 = 4
        subplot_n1 = 5
        subplot_n2 = 6

    plt.subplot(2, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    if epsilon == 0:
        plt.text(-0.3, 1, "Brent’s method on a quadratic function:", fontweight='bold', horizontalalignment='left',
            fontsize=12)
        caption_text = "This converges in 3 iterations, as the quadratic \napproximation is then exact."
        plt.text(-0.3, 0.83, caption_text,
            horizontalalignment='left',
            fontsize=12,
            wrap=True)
    else:
        plt.text(-0.3, 1, "Brent’s method on a non-convex function", fontweight='bold', horizontalalignment='left',
            fontsize=12)
        caption_text = "Note that the fact that the optimizer avoided\nthe local minimum is a matter of luck."
        plt.text(-0.3, 0.83, caption_text,
            horizontalalignment='left',
            fontsize=12,
            wrap=True)

    plt.subplot(2, 3, subplot_n1)

    # A convex function
    plt.plot(x, f(x), linewidth=2)

    # Apply brent method. To have access to the iteration, do this in an
    # artificial way: allow the algorithm to iter only once
    all_x = list()
    all_y = list()
    for iter in range(30):
        result = optimize.minimize_scalar(f, bracket=(-5, 2.9, 4.5), method="Brent",
                    options={"maxiter": iter}, tol=np.finfo(1.).eps)
        if result.success:
            print('Converged at ', iter)
            break

        this_x = result.x
        all_x.append(this_x)
        all_y.append(f(this_x))
        if iter < 6:
            plt.text(this_x - .05*np.sign(this_x) - .05,
                    f(this_x) + 1.2*(.3 - iter % 2), iter + 1,
                    size=12)

    plt.plot(all_x[:10], all_y[:10], 'k+', markersize=12, markeredgewidth=2)
    plt.plot(all_x[-1], all_y[-1], 'rx', markersize=12)
    plt.ylim(ymin=-1, ymax=8)

    plt.subplot(2, 3, subplot_n2)
    plt.semilogy(np.abs(all_y - all_y[-1]), linewidth=2)
    plt.ylabel('Error on f(x)')
    plt.xlabel('Iteration')

plt.tight_layout()

**Start of note**
You can use different solvers using the parameter `method`.
**End of note**

**Start of note**
{func}`scipy.optimize.minimize_scalar` can also be used for optimization
constrained to an interval using the parameter `bounds`.
**End of note**


### Gradient based methods

#### Some intuitions about gradient descent

Here we focus on **intuitions**, not code. Code will follow.

[Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)
basically consists in taking small steps in the direction of the
gradient, that is the direction of the *steepest descent*.

In [ ]:
x_min, x_max = -1, 2
y_min, y_max = 2.25 / 3 * x_min - 0.2, 2.25 / 3 * x_max - 0.2

levels = {}

plt.figure(figsize=(10, 6))
plt.title('Fixed step gradient descent', fontweight='bold')
plt.axis('off')
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (   (mk_quad(0.7), gradient_descent),
        (mk_quad(0.02), gradient_descent),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    titles = ["A well-conditioned quadratic function.",
              "An ill-conditioned quadratic function."]

    captions = [ "",
    "The core problem of gradient-methods on\n ill-conditioned problems is that the gradient\ntends not to point in the direction of the\nminimum"
      ]

    plt.subplot(2, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    plt.text(-0.3, 1, titles[index], fontweight='bold', horizontalalignment='left',
        fontsize=12)
    caption_text = captions[index]
    plt.text(-0.3, 0.6, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(2, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(2, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

We can see that very anisotropic ([ill-conditioned](https://en.wikipedia.org/wiki/Condition_number)) functions are harder
to optimize.

**Start of admonition: Take home message: conditioning number and preconditioning**
If you know natural scaling for your variables, prescale them so that
they behave similarly. This is related to [preconditioning](https://en.wikipedia.org/wiki/Preconditioner).
**End of admonition**

Also, it clearly can be advantageous to take bigger steps. This
is done in gradient descent code using a
[line search](https://en.wikipedia.org/wiki/Line_search).

In [ ]:
x_min, x_max = -1, 2
y_min, y_max = 2.25 / 3 * x_min - 0.2, 2.25 / 3 * x_max - 0.2

levels = {}

plt.figure(figsize=(10, 10))
plt.title('Adaptive step gradient descent', fontweight='bold')
plt.axis('off')
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        (mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        (mk_quad(0.02), gradient_descent_adaptative),
        (mk_gauss(0.02), gradient_descent_adaptative),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
            gradient_descent_adaptative,),
        #(mk_gauss(0.02), conjugate_gradient),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        #(mk_quad(0.02), newton_cg),
        #(mk_gauss(0.02), newton_cg),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        #(mk_quad(0.02), bfgs),
        #(mk_gauss(0.02), bfgs),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        #(mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        #(mk_gauss(0.02), nelder_mead),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    # titles = []

    captions = ["A well-conditioned quadratic function.",
                "An ill-conditioned quadratic function.",
                "An ill-conditioned non-quadratic function.",
                "An ill-conditioned very non-quadratic function."]

    plt.subplot(4, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    #plt.text(-0.3, 1, titles[row], fontweight='bold', horizontalalignment='left',
        #fontsize=12)
    caption_text = captions[row-1]
    plt.text(-0.3, 0.83, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(4, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(4, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

The more a function looks like a quadratic function (elliptic
iso-curves), the easier it is to optimize.

#### Conjugate gradient descent

The gradient descent algorithms above are toys not to be used on real
problems.

As can be seen from the above experiments, one of the problems of the
simple gradient descent algorithms, is that it tends to oscillate across
a valley, each time following the direction of the gradient, that makes
it cross the valley. The conjugate gradient solves this problem by adding
a *friction* term: each step depends on the two last values of the
gradient and sharp turns are reduced.

In [ ]:
x_min, x_max = -1, 2
y_min, y_max = 2.25 / 3 * x_min - 0.2, 2.25 / 3 * x_max - 0.2

levels = {}

plt.figure(figsize=(12, 6))
plt.title('Conjugate gradient descent', fontweight='bold')
plt.axis('off')
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        #(mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        #(mk_quad(0.02), gradient_descent_adaptative),
        #(mk_gauss(0.02), gradient_descent_adaptative),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
            #gradient_descent_adaptative,),
        (mk_gauss(0.02), conjugate_gradient),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        #(mk_quad(0.02), newton_cg),
        #(mk_gauss(0.02), newton_cg),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        #(mk_quad(0.02), bfgs),
        #(mk_gauss(0.02), bfgs),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        #(mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        #(mk_gauss(0.02), nelder_mead),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    # titles = []

    captions = ["A well-conditioned quadratic function.",
                "An ill-conditioned quadratic function.",
                "An ill-conditioned non-quadratic function.",
                "An ill-conditioned very non-quadratic function."]

    plt.subplot(2, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    #plt.text(-0.3, 1, titles[row], fontweight='bold', horizontalalignment='left',
        #fontsize=12)
    caption_text = captions[row-1]
    plt.text(-0.3, 0.83, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(2, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(2, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

SciPy provides {func}`scipy.optimize.minimize` to find the minimum of scalar
functions of one or more variables. The simple conjugate gradient method can
be used by setting the parameter `method` to CG

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
sp.optimize.minimize(f, [2, -1], method="CG")

Gradient methods need the Jacobian (gradient) of the function. They can compute it
numerically, but will perform better if you can pass them the gradient:

In [ ]:
def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
sp.optimize.minimize(f, [2, 1], method="CG", jac=jacobian)

Note that the function has only been evaluated 27 times, compared to 108
without the gradient.

### Newton and quasi-newton methods

#### Newton methods: using the Hessian (2nd differential)

[Newton methods](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization) use a
local quadratic approximation to compute the jump direction. For this
purpose, they rely on the 2 first derivative of the function: the
*gradient* and the [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix).

In [ ]:
levels = {}

plt.figure(figsize=(12, 8))
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        #(mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        #(mk_quad(0.02), gradient_descent_adaptative),
        #(mk_gauss(0.02), gradient_descent_adaptative),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
           # gradient_descent_adaptative,),
        #(mk_gauss(0.02), conjugate_gradient),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        (mk_quad(0.02), newton_cg),
        (mk_gauss(0.02), newton_cg),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        #(mk_quad(0.02), bfgs),
        #(mk_gauss(0.02), bfgs),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        #(mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        #(mk_gauss(0.02), nelder_mead),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    titles = ["An ill-conditioned quadratic function:",
             "An ill-conditioned quadratic function:",
              "An ill-conditioned very non-quadratic \nfunction:"]

    captions = ["Note that, as the quadratic\napproximation is exact, the Newton\nmethod is blazing fast",
               "Here we are optimizing a\nGaussian, which is always below\nits quadratic approximation. As a\nresult, the Newton method \novershoots and leads to oscillations.",
               ""]

    plt.subplot(3, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    plt.text(-0.3, 1, titles[row-1], fontweight='bold', horizontalalignment='left',
        fontsize=12)
    caption_text = captions[row-1]
    plt.text(-0.3, 0.5, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(3, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(3, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

In SciPy, you can use the Newton method by setting `method` to Newton-CG in
{func}`scipy.optimize.minimize`. Here, CG refers to the fact that an internal
inversion of the Hessian is performed by conjugate gradient.

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
sp.optimize.minimize(f, [2,-1], method="Newton-CG", jac=jacobian)

Note that compared to a conjugate gradient (above), Newton's method has
required less function evaluations, but more gradient evaluations, as it
uses it to approximate the Hessian. Let's compute the Hessian and pass it
to the algorithm:

In [ ]:
def hessian(x): # Computed with sympy
    return np.array(((1 - 4*x[1] + 12*x[0]**2, -4*x[0]), (-4*x[0], 2)))

sp.optimize.minimize(f, [2,-1], method="Newton-CG", jac=jacobian, hess=hessian)

**Start of note**
At very high-dimension, the inversion of the Hessian can be costly
and unstable (large scale > 250).
**End of note**

**Start of note**
Newton optimizers should not to be confused with Newton's root finding
method, based on the same principles, {func}`scipy.optimize.newton`.
**End of note**
#### Quasi-Newton methods: approximating the Hessian on the fly

**BFGS**: BFGS (Broyden-Fletcher-Goldfarb-Shanno algorithm) refines at
each step an approximation of the Hessian.

## Full code examples

In [ ]:
levels = {}

plt.figure(figsize=(12, 8))
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        #(mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        #(mk_quad(0.02), gradient_descent_adaptative),
        #(mk_gauss(0.02), gradient_descent_adaptative),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
           # gradient_descent_adaptative,),
        #(mk_gauss(0.02), conjugate_gradient),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        #(mk_quad(0.02), newton_cg),
        #(mk_gauss(0.02), newton_cg),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        (mk_quad(0.02), bfgs),
        (mk_gauss(0.02), bfgs),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        #(mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        #(mk_gauss(0.02), nelder_mead),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    titles = ["An ill-conditioned quadratic function:",
              "An ill-conditioned non-quadratic function:",
              "An ill-conditioned very non-quadratic function:"]

    captions = ["\nAn ill-conditioned quadratic function: On an \nexactly quadratic function, BFGS is not as fast\nas Newton’s method, but still very fast.",
                "\n\nHere BFGS does better than Newton, as its\nempirical estimate of the curvature is better than\nthat given by the Hessian.",
                ""]

    plt.subplot(3, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    plt.text(-0.3, 1, titles[index], fontweight='bold', horizontalalignment='left',
        fontsize=12)
    caption_text = captions[index]
    plt.text(-0.3, 0.7, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(3, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(3, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
sp.optimize.minimize(f, [2, -1], method="BFGS", jac=jacobian)

**L-BFGS:** Limited-memory BFGS Sits between BFGS and conjugate gradient:
in very high dimensions (> 250) the Hessian matrix is too costly to
compute and invert. L-BFGS keeps a low-rank version. In addition, box bounds
are also supported by L-BFGS-B:

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2

def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))

sp.optimize.minimize(f, [2, 2], method="L-BFGS-B", jac=jacobian)

### Gradient-less methods

#### A shooting method: the Powell algorithm

Almost a gradient approach:

In [ ]:
levels = {}

plt.figure(figsize=(12, 6))
plt.title("Powell's method", fontweight='bold')
plt.axis('off')
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        #(mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        #(mk_quad(0.02), gradient_descent_adaptative),
        #(mk_gauss(0.02), gradient_descent_adaptative),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
            #gradient_descent_adaptative,),
        #(mk_gauss(0.02), conjugate_gradient),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        #(mk_quad(0.02), newton_cg),
        #(mk_gauss(0.02), newton_cg),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        #(mk_quad(0.02), bfgs),
        #(mk_gauss(0.02), bfgs),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        (mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        #(mk_gauss(0.02), nelder_mead),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1
    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    titles = ["An ill-conditioned quadratic function:",
             "An ill-conditioned very non-quadratic function:"]

    captions = ["Powell’s method isn’t too sensitive to local \nill-conditionning in low dimensions.",
                ""]

    plt.subplot(2, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    plt.text(-0.3, 1, titles[index], fontweight='bold', horizontalalignment='left',
        fontsize=12)
    caption_text = captions[index]
    plt.text(-0.3, 0.83, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(2, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(2, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

#### Simplex method: the Nelder-Mead

The Nelder-Mead algorithms is a generalization of dichotomy approaches to
high-dimensional spaces. The algorithm works by refining a [simplex](https://en.wikipedia.org/wiki/Simplex), the generalization of intervals
and triangles to high-dimensional spaces, to bracket the minimum.

**Strong points**: it is robust to noise, as it does not rely on
computing gradients. Thus it can work on functions that are not locally
smooth such as experimental data points, as long as they display a
large-scale bell-shape behavior. However it is slower than gradient-based
methods on smooth, non-noisy functions.

In [ ]:
levels = {}

plt.figure(figsize=(12, 6))
plt.title("Simplex method: the Nelder-Mead", fontweight='bold')
plt.axis('off')
for index, ((f, f_prime, hessian), optimizer) in enumerate(
    (
        #(mk_quad(0.7), gradient_descent),
        #(mk_quad(0.7), gradient_descent_adaptative),
        #(mk_quad(0.02), gradient_descent),
        #(mk_quad(0.02), gradient_descent_adaptative),
        #(mk_gauss(0.02), gradient_descent_adaptative),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian),
            #gradient_descent_adaptative,),
        #(mk_gauss(0.02), conjugate_gradient),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), conjugate_gradient),
        #(mk_quad(0.02), newton_cg),
        #(mk_gauss(0.02), newton_cg),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), newton_cg),
        #(mk_quad(0.02), bfgs),
        #(mk_gauss(0.02), bfgs),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), bfgs),
        #(mk_quad(0.02), powell),
        #(mk_gauss(0.02), powell),
        #((rosenbrock, rosenbrock_prime, rosenbrock_hessian), powell),
        (mk_gauss(0.02), nelder_mead),
        ((rosenbrock, rosenbrock_prime, rosenbrock_hessian), nelder_mead),
    )
):
    # Compute a gradient-descent
    x_i, y_i = 1.6, 1.1
    counting_f_prime = CountingFunction(f_prime)
    counting_hessian = CountingFunction(hessian)
    logging_f = LoggingFunction(f, counter=counting_f_prime.counter)
    all_x_i, all_y_i, all_f_i = optimizer(
        np.array([x_i, y_i]), logging_f, counting_f_prime, hessian=counting_hessian
    )

    row = index+1

    subplot_n0, subplot_n1, subplot_n2 = get_subplot_n(index)

    titles = ["An ill-conditioned non-quadratic function:",
             "An ill-conditioned very non-quadratic function:"]

    captions = ["",
                ""]

    plt.subplot(2, 3, subplot_n0)
    plt.scatter([0, 1], [0, 1], c='white')
    plt.axis('off')
    plt.text(-0.3, 1, titles[index], fontweight='bold', horizontalalignment='left',
        fontsize=12)
    caption_text = captions[index]
    plt.text(-0.3, 0.83, caption_text,
        horizontalalignment='left',
        fontsize=12,
        wrap=True)

    if not max(all_y_i) < y_max:
        x_min *= 1.2
        x_max *= 1.2
        y_min *= 1.2
        y_max *= 1.2
    x, y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
    x = x.T
    y = y.T

    X = np.concatenate((x[np.newaxis, ...], y[np.newaxis, ...]), axis=0)
    z = np.apply_along_axis(f, 0, X)
    log_z = np.log(z + 0.01)

    plt.subplot(2, 3, subplot_n1)
    plt.imshow(
        log_z,
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gray_r,
        origin="lower",
        vmax=log_z.min() + 1.5 * np.ptp(log_z),
    )
    contours = plt.contour(
        log_z,
        levels=levels.get(f),
        extent=[x_min, x_max, y_min, y_max],
        cmap=plt.cm.gnuplot,
        origin="lower",
    )
    levels[f] = contours.levels
    plt.clabel(contours, inline=1, fmt=super_fmt, fontsize=14)

    plt.plot(all_x_i, all_y_i, "b-", linewidth=2)
    plt.plot(all_x_i, all_y_i, "k+")

    plt.plot(logging_f.all_x_i, logging_f.all_y_i, "k.", markersize=2)

    plt.plot([0], [0], "rx", markersize=12)

    plt.xticks(())
    plt.yticks(())
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.subplot(2, 3, subplot_n2)
    plt.semilogy(np.maximum(np.abs(all_f_i), 1e-30), linewidth=2, label="# iterations")
    plt.ylabel("Error on f(x)")
    plt.semilogy(
        logging_f.counts,
        np.maximum(np.abs(logging_f.all_f_i), 1e-30),
        linewidth=2,
        color="g",
        label="# function calls",
    )
    plt.legend(
        loc="upper right",
        frameon=True,
        prop={"size": 11},
        borderaxespad=0,
        handlelength=1.5,
        handletextpad=0.5,
    )
plt.tight_layout()

Using the Nelder-Mead solver in {func}`scipy.optimize.minimize`:

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
sp.optimize.minimize(f, [2, -1], method="Nelder-Mead")

### Global optimizers

If your problem does not admit a unique local minimum (which can be hard
to test unless the function is convex), and you do not have prior
information to initialize the optimization close to the solution, you
may need a global optimizer.

#### Brute force: a grid search

{func}`scipy.optimize.brute` evaluates the function on a given grid of
parameters and returns the parameters corresponding to the minimum
value. The parameters are specified with ranges given to
{obj}`numpy.mgrid`. By default, 20 steps are taken in each direction:

In [ ]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
sp.optimize.brute(f, ((-1, 2), (-1, 2)))

## Practical guide to optimization with SciPy

### Choosing a method

All methods are exposed as the `method` argument of
{func}`scipy.optimize.minimize`.

<!---
![](auto_examples/images/sphx_glr_plot_compare_optimizers_001.png)
# For parsing.
:align: center
:width: 95%
-->

In [ ]:
"""
Plotting the comparison of optimizers
======================================

Plots the results from the comparison of optimizers.

"""

import pickle
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

results = pickle.load(
    open(f"helper/compare_optimizers_py{sys.version_info[0]}.pkl", "rb")
)
n_methods = len(list(results.values())[0]["Rosenbrock  "])
n_dims = len(results)

symbols = "o>*Ds"

plt.figure(1, figsize=(10, 4))
plt.clf()

nipy_spectral = matplotlib.colormaps["nipy_spectral"]
colors = nipy_spectral(np.linspace(0, 1, n_dims))[:, :3]

method_names = list(list(results.values())[0]["Rosenbrock  "].keys())
method_names.sort(key=lambda x: x[::-1], reverse=True)

for n_dim_index, ((n_dim, n_dim_bench), color) in enumerate(
    zip(sorted(results.items()), colors, strict=True)
):
    for (cost_name, cost_bench), symbol in zip(
        sorted(n_dim_bench.items()), symbols, strict=True
    ):
        for (
            method_index,
            method_name,
        ) in enumerate(method_names):
            this_bench = cost_bench[method_name]
            bench = np.mean(this_bench)
            plt.semilogy(
                [
                    method_index + 0.1 * n_dim_index,
                ],
                [
                    bench,
                ],
                marker=symbol,
                color=color,
            )

# Create a legend for the problem type
for cost_name, symbol in zip(sorted(n_dim_bench.keys()), symbols, strict=True):
    plt.semilogy(
        [
            -10,
        ],
        [
            0,
        ],
        symbol,
        color=".5",
        label=cost_name,
    )

plt.xticks(np.arange(n_methods), method_names, size=11)
plt.xlim(-0.2, n_methods - 0.5)
plt.legend(loc="best", numpoints=1, handletextpad=0, prop={"size": 12}, frameon=False)
plt.ylabel("# function calls (a.u.)")

# Create a second legend for the problem dimensionality
plt.twinx()

for n_dim, color in zip(sorted(results.keys()), colors, strict=True):
    plt.plot(
        [
            -10,
        ],
        [
            0,
        ],
        "o",
        color=color,
        label=f"# dim: {n_dim}",
    )
plt.legend(
    loc=(0.47, 0.07),
    numpoints=1,
    handletextpad=0,
    prop={"size": 12},
    frameon=False,
    ncol=2,
)
plt.xlim(-0.2, n_methods - 0.5)

plt.xticks(np.arange(n_methods), method_names)
plt.yticks(())

plt.tight_layout()

**With knowledge of the gradient**

* **BFGS** or **L-BFGS**.
* Computational overhead of BFGS is larger than that L-BFGS, itself
  larger than that of conjugate gradient. On the other side, BFGS usually
  needs less function evaluations than CG. Thus conjugate gradient method
  is better than BFGS at optimizing computationally cheap functions.

**With the Hessian**

* If you can compute the Hessian, prefer the Newton method
  (**Newton-CG** or **TCG**).

**If you have noisy measurements**

* Use **Nelder-Mead** or **Powell**.

### Making your optimizer faster

- Choose the right method (see above), do compute analytically the
  gradient and Hessian, if you can.
- Use [preconditionning](https://en.wikipedia.org/wiki/Preconditioner)
  when possible.
- Choose your initialization points wisely. For instance, if you are
  running many similar optimizations, warm-restart one with the results of
  another.
- Relax the tolerance if you don't need precision using the parameter `tol`.

### Computing gradients

Computing gradients, and even more Hessians, is very tedious but worth
the effort. Symbolic computation with {ref}`Sympy <sympy>` may come in
handy.

**Warning**

A *very* common source of optimization not converging well is human
error in the computation of the gradient. You can use
{func}`scipy.optimize.check_grad` to check that your gradient is
correct. It returns the norm of the different between the gradient
given, and a gradient computed numerically:

In [ ]:
sp.optimize.check_grad(f, jacobian, [2, -1])

See also {func}`scipy.optimize.approx_fprime` to find your errors.

### Synthetic exercises

**A simple (?) quadratic function**

**Start of exercise**

Optimize the following function, using K[0] as a starting point:

In [ ]:
rng = np.random.default_rng(27446968)
K = rng.normal(size=(100, 100))

def f(x):
    return np.sum((K @ (x - 1))**2) + np.sum(x**2)**2

Time your approach. Find the fastest approach. Why is BFGS not
working well?

**End of exercise**

**See the [corresponding page](/scipy-lecture-notes/advanced/mathematical_optimization/index.html) for solution**

**A locally flat minimum**

**Start of exercise**

Consider the function `exp(-1/(.1*x**2 + y**2)`. This function admits
a minimum in (0, 0). Starting from an initialization at (1, 1), try
to get within 1e-8 of this minimum point.

This exercise is hard because the function is very flat around the minimum
(all its derivatives are zero). Thus gradient information is unreliable.

**End of exercise**

**See the [corresponding page](/scipy-lecture-notes/advanced/mathematical_optimization/index.html) for solution**

## Special case: non-linear least-squares

### Minimizing the norm of a vector function

Least square problems, minimizing the norm of a vector function, have a
specific structure that can be used in the [Levenberg–Marquardt algorithm](https://en.wikipedia.org/wiki/Levenberg-Marquardt_algorithm)
implemented in {func}`scipy.optimize.leastsq`.

Lets try to minimize the norm of the following vectorial function:

In [ ]:
def f(x):
    return np.arctan(x) - np.arctan(np.linspace(0, 1, len(x)))

In [ ]:
x0 = np.zeros(10)
sp.optimize.leastsq(f, x0)

This took 67 function evaluations (check it with 'full_output=True'). What
if we compute the norm ourselves and use a good generic optimizer (BFGS):

In [ ]:
def g(x):
    return np.sum(f(x)**2)
result = sp.optimize.minimize(g, x0, method="BFGS")
result.fun

BFGS needs more function calls, and gives a less precise result.

**Start of note**
`leastsq` is interesting compared to BFGS only if the
dimensionality of the output vector is large, and larger than the number
of parameters to optimize.
**End of note**

**Start of warning**
If the function is linear, this is a linear-algebra problem, and
should be solved with {func}`scipy.linalg.lstsq`.
**End of warning**

### Curve fitting

Least square problems occur often when fitting a non-linear to data.
While it is possible to construct our optimization problem ourselves,
SciPy provides a helper function for this purpose:
{func}`scipy.optimize.curve_fit`:

In [ ]:
def f(t, omega, phi):
    return np.cos(omega * t + phi)

In [ ]:
x = np.linspace(0, 3, 50)
rng = np.random.default_rng(27446968)
y = f(x, 1.5, 1) + .1*rng.normal(size=50)

In [ ]:
sp.optimize.curve_fit(f, x, y)

In [ ]:
rng = np.random.default_rng(27446968)


# Our test function
def f(t, omega, phi):
    return np.cos(omega * t + phi)


# Our x and y data
x = np.linspace(0, 3, 50)
y = f(x, 1.5, 1) + 0.1 * np.random.normal(size=50)

# Fit the model: the parameters omega and phi can be found in the
# `params` vector
params, params_cov = sp.optimize.curve_fit(f, x, y)

# plot the data and the fitted curve
t = np.linspace(0, 3, 1000)

plt.plot(x, y, "bx")
plt.plot(t, f(t, *params), "r-");

**Start of exercise**

Do the same with omega = 3. What is the difficulty?

**End of exercise**

## Optimization with constraints

### Box bounds

Box bounds correspond to limiting each of the individual parameters of
the optimization. Note that some problems that are not originally written
as box bounds can be rewritten as such via change of variables. Both
{func}`scipy.optimize.minimize_scalar` and {func}`scipy.optimize.minimize`
support bound constraints with the parameter `bounds`:

In [ ]:
def f(x):
    return np.sqrt((x[0] - 3)**2 + (x[1] - 2)**2)

sp.optimize.minimize(f, np.array([0, 0]), bounds=((-1.5, 1.5), (-1.5, 1.5)))

In [ ]:
x, y = np.mgrid[-2.9:5.8:0.05, -2.5:5:0.05]  # type: ignore[misc]
x = x.T
y = y.T

for i in (1, 2):
    # Create 2 figure: only the second one will have the optimization
    # path
    if i == 2:
        plt.figure(i, figsize=(3, 2.5))
        plt.clf()
        plt.axes((0, 0, 1, 1))

        contours = plt.contour(
            np.sqrt((x - 3) ** 2 + (y - 2) ** 2),
            extent=[-3, 6, -2.5, 5],
            cmap="gnuplot",
        )
        plt.clabel(contours, inline=1, fmt="%1.1f", fontsize=14)
        plt.plot(
            [-1.5, -1.5, 1.5, 1.5, -1.5], [-1.5, 1.5, 1.5, -1.5, -1.5], "k", linewidth=2
        )
        plt.fill_between([-1.5, 1.5], [-1.5, -1.5], [1.5, 1.5], color=".8")
        plt.axvline(0, color="k")
        plt.axhline(0, color="k")

        plt.text(-0.9, 4.4, "$x_2$", size=20)
        plt.text(5.6, -0.6, "$x_1$", size=20)
        plt.axis("equal")
        plt.axis("off")

# And now plot the optimization path
accumulator = []


def f(x):
    # Store the list of function calls
    accumulator.append(x)
    return np.sqrt((x[0] - 3) ** 2 + (x[1] - 2) ** 2)


# We don't use the gradient, as with the gradient, L-BFGS is too fast,
# and finds the optimum without showing us a pretty path
def f_prime(x):
    r = np.sqrt((x[0] - 3) ** 2 + (x[0] - 2) ** 2)
    return np.array(((x[0] - 3) / r, (x[0] - 2) / r))


sp.optimize.minimize(
    f, np.array([0, 0]), method="L-BFGS-B", bounds=((-1.5, 1.5), (-1.5, 1.5))
)

accumulated = np.array(accumulator)
plt.plot(accumulated[:, 0], accumulated[:, 1]);

<!---
![](auto_examples/images/sphx_glr_plot_constraints_002.png)
# For parsing.
:align: right
:scale: 75%
:target: auto_examples/plot_constraints.html
-->

### General constraints

Equality and inequality constraints specified as functions: $f(x) = 0$
and $g(x) < 0$.

#### {func}`scipy.optimize.fmin_slsqp` Sequential least square programming:
equality and inequality constraints:

<!---
![](auto_examples/images/sphx_glr_plot_non_bounds_constraints_001.png)
# For parsing.
:align: right
:scale: 75%
:target: auto_examples/plot_non_bounds_constraints.html
-->

In [ ]:
def f(x):
    return np.sqrt((x[0] - 3)**2 + (x[1] - 2)**2)

In [ ]:
def constraint(x):
    return np.atleast_1d(1.5 - np.sum(np.abs(x)))

In [ ]:
x0 = np.array([0, 0])
sp.optimize.minimize(f, x0, constraints={"fun": constraint, "type": "ineq"})

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

x, y = np.mgrid[-2.03:4.2:0.04, -1.6:3.2:0.04]  # type: ignore[misc]
x = x.T
y = y.T

plt.figure(1, figsize=(3, 2.5))
plt.clf()
plt.axes((0, 0, 1, 1))

contours = plt.contour(
    np.sqrt((x - 3) ** 2 + (y - 2) ** 2),
    extent=[-2.03, 4.2, -1.6, 3.2],
    cmap="gnuplot",
)
plt.clabel(contours, inline=1, fmt="%1.1f", fontsize=14)
plt.plot([-1.5, 0, 1.5, 0, -1.5], [0, 1.5, 0, -1.5, 0], "k", linewidth=2)
plt.fill_between([-1.5, 0, 1.5], [0, -1.5, 0], [0, 1.5, 0], color=".8")
plt.axvline(0, color="k")
plt.axhline(0, color="k")

plt.text(-0.9, 2.8, "$x_2$", size=20)
plt.text(3.6, -0.6, "$x_1$", size=20)
plt.axis("tight")
plt.axis("off")

# And now plot the optimization path
accumulator = []


def f(x):
    # Store the list of function calls
    accumulator.append(x)
    return np.sqrt((x[0] - 3) ** 2 + (x[1] - 2) ** 2)


def constraint(x):
    return np.atleast_1d(1.5 - np.sum(np.abs(x)))


sp.optimize.minimize(
    f, np.array([0, 0]), method="SLSQP", constraints={"fun": constraint, "type": "ineq"}
)

accumulated = np.array(accumulator)
plt.plot(accumulated[:, 0], accumulated[:, 1]);

**Start of warning**
The above problem is known as the [Lasso](<https://en.wikipedia.org/wiki/Lasso_(statistics)>)
problem in statistics, and there exist very efficient solvers for it
(for instance in [scikit-learn](https://scikit-learn.org)). In
general do not use generic solvers when specific ones exist.
**End of warning**

**Start of admonition: Lagrange multipliers**
If you are ready to do a bit of math, many constrained optimization
problems can be converted to non-constrained optimization problems
using a mathematical trick known as [Lagrange multipliers](https://en.wikipedia.org/wiki/Lagrange_multiplier).
**End of admonition**

## Full code examples

<!---
include the gallery. Skip the first line to avoid the "orphan"
declaration
-->

**Start of admonition: See also**

**Other Software**

SciPy tries to include the best well-established, general-use,
and permissively-licensed optimization algorithms available. However,
even better options for a given task may be available in other libraries;
please also see [IPOPT] and [PyGMO].
**End of admonition**

[ipopt]: https://github.com/xuy/pyipopt
[pygmo]: https://esa.github.io/pygmo2/